## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.functional import gelu
from torch.nn import CrossEntropyLoss

from datasets import load_dataset, DatasetDict

from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from transformers import (RobertaTokenizer, PreTrainedModel, RobertaConfig, 
                          RobertaForMaskedLM, DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

from transformers.modeling_outputs import MaskedLMOutput

## Helper functions

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

## Paths

In [ ]:
data = Path("data/")
!ls {data}

oscar.eo.ds  oscar.eo.txt


In [ ]:
model_dir = "models/esperberto"
!ls {model_dir}

merges.txt  vocab.json


## Get data

In [ ]:
!wget -c -O data/oscar.eo.txt https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2021-04-16 09:19:52--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.84.114.112, 99.84.114.24, 99.84.114.120, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.84.114.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘data/oscar.eo.txt’

data/oscar.eo.txt   100%[===================>] 298.25M  56.4MB/s    in 5.3s    

2021-04-16 09:19:58 (56.3 MB/s) - ‘data/oscar.eo.txt’ saved [312733741/312733741]



In [ ]:
!head {data/"oscar.eo.txt"}

Ĉu ... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari Diservon
Temas pri kolekto de kristanaj kantoj, eldonita de Adolf Burkhardt inter 1974 kaj 1990 en dek kajeretoj. Ili estas reeldonitaj inter 1995 kaj 1998 de Bernhard Eichkorn en tri kajeroj, kies tria estas pliampleksigita per Dek Novaj Kantoj kaj suplemento, same de Adolf Burkhardt.
En la dua kaj tria kajero oni adiciis 300 al la originaj kantonumeroj, por ke oni povu pli facile uzi la kajerojn kune kun la KELI-himnaro Adoru Kantante, kiu havas malpli ol 300 numerojn.
Ni ĝojus, se iu trovus bonajn ekzemplerojn de la dek originaj kajeretoj kaj tempon por skani ankaŭ ilin. Bonvolu ekkontaktiĝi kun ni!
Lerni Esperanton per telefono, novaĵoj Poŝtkarto 120 jaroj de fervojo Svitavy-Polička 189… T.n.migranta poŝtkarto el 1908 BK - Kongresa Biblioteko en Vaŝingtono 1- 910 BK - Nederlando- Esperanta elektra tramo en Hago (… La lernolibro "Esperanto per rekta metodo" jam en… IMG 7181 Nova poŝtkar

## Train tokenizer

In [ ]:
paths = [str(x) for x in data.glob("**/*.txt")]
paths

['data/oscar.eo.txt']

In [ ]:
tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
tokenizer.save_model(model_dir)

['models/esperberto/vocab.json', 'models/esperberto/merges.txt']

In [ ]:
tokenizer = ByteLevelBPETokenizer(
    f"{model_dir}/vocab.json",
    f"{model_dir}/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Mi estas Julien.")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("Mi estas Julien.").tokens

['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

## Load data

In [ ]:
ds = load_dataset('text', data_files={'train': [paths[0]]})
ds

Using custom data configuration default-31220d7f73477105
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-31220d7f73477105/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 974616
    })
})

In [ ]:
show_random_elements(ds["train"])

,text
0,La libera direktisto de pasvorto kaj sekretaj datumoj. La programaro uzas specialan ĉifrado algoritmo por la confidencialidad de la informo stokita.
1,"18 sed memoru la Eternulon, vian Dion, ĉar Li estas Tiu, kiu donas al vi forton, por akiri grandan havon, por plenumi Sian interligon, pri kiu Li ĵuris al viaj patroj, kiel vi vidas nun. 19 Kaj mi avertas vin hodiaŭ, ke, se vi forgesos la Eternulon, vian Dion, kaj sekvos aliajn diojn kaj servos al ili kaj kliniĝos antaŭ ili, tiam vi pereos; 20 simile al la popoloj, kiujn la Eternulo pereigas antaŭ vi, tiel vi pereos; pro tio, ke vi ne obeos la voĉon de la Eternulo, via Dio."
2,"Asquith sekvis tion per jesado teni Komisionojn de Enketo en la konduton de Dardaneloj kaj da la Mesopotamian kampanjo, kie Aliancite fortoj estis devigita kapitulaci ĉe Kut. [273] Sir Maurice Hankey, Sekretario al la Milito-Komisiono, pripensis tion; ""la koalicio neniam resaniĝis. Dum (ĝia) lasta kvin monatoj, la funkcio de la Ĉefkomando estis aranĝita sub la ombron de tiuj mortenketoj."" [274] Sed tiuj eraroj estis ombritaj fare de la limigita progreso kaj enormaj viktimoj de la Batalo ĉe la Somme, kiu komenciĝis la 1an de julio 1916, kaj tiam per alia giganta persona perdo, la morto de la filo de Asquith Raymond, la 15an de septembro ĉe la Battle of Flers-Courcelette (Batalo de Flers-Courcelette). [275] La rilato de Asquith kun lia majoratulo ne estis facila. Raymond skribis al sia edzino frue en 1916; ""Ĉu Margot-babiladoj plu babilaĵo al vi pri la malhomeco de ŝia paŝinfanoj vi povas maldaŭrigi ŝian buŝon rakontante al ŝi ke dum mia 10 monatekzilo ĉi tie la Pm neniam skribis al mi linion de iu priskribo."" [276] Sed la morto de Raymond estis frakasa, Viola skribo; ""... por vidi Patro-sufero tiel tordas tian"", [277] kaj Asquith pasigis multon da la sekvaj monatoj ""malparolema kaj malfacila alproksimiĝi"". [278] La Milito alportis neniun libertempon, Churchill skribanta tion; ""La malsukceso rompi la germanan linion en la Somme, la reakiro da la ĝermanaj potencoj en la orienta [i.e. la malvenko de la Brusilov Ofensivo], la ruino de Rumanio kaj la komencoj de renoviĝinta submarŝipa milito fortigis kaj stimulis ĉiujn tiujn fortojn kiuj insistis sur daŭre pli granda vigleco en la konduto de aferoj."""
3,"fotaro de Juci (Vidor Judit) estas cxe: http://jucimam.multiply.com/ fotaro de Maria Nuyanzina estas cxe http://www.flickr.com/, fotaro de Jxomart kaj Natasxa estas cxe http://www.jomart.net/bilder/ kaj fotaro de Rade Kuzmanovic estas cxi tie: http://ijs7.aim.ac.yu/"
4,"La suba korpo, kun naŭ truoj kaj ok klavoj, tenata de la dekstra mano (foje pli por la basaj tipoj);"
5,"Petro la kultinda, kiu tradukigis la Koranon kaj la Talmudon kaj studis ilin, estis verŝajne la unua kontraŭjudulo ; li skribis ideojn kiuj utilis, en Liono, dum la kunveno (Koncilio)... ,kiu organizis inkvizicion - terura afero : arestado kaj bruligo de ne-katolikaj homoj. Abato Petro mortis en 1156."
6,"La agado de UEA sur E-ista kampo estis tiu de ligilo inter la ankoraŭ funkciantaj organizaĵoj en kelkaj neŭtralaj landoj kaj la aktivaj E-istoj. Aperis regule ,E‘ ĉiumonate kun artikoloj pri tutmondaj problemoj. En 1916 aperis jarlibro, kiu enhavis ĉefajn informojn kaj la adresaron de delegitoj. Plie aperis dufoje libreto: „E dum la milito“, kiu enhavis resumon de la stato de la movado. Elokvente parolis pri la sekvoj de la milito la ciferoj. En 1914 la nombro de pagintaj anoj estis 7233, en 1915 estis 2699, en 1918 1958."
7,"Pri la aktorino Z. oni diris, ke ŝi mortigis sin pro malfeliĉa amo. Sinjoro Kojno diris: ŝi mortigis sin pro amo al si mem. La X-on ŝi ĉiukaze ne povas esti aminta. Alie ŝi tion ne estus farinta al li. Amo estas la deziro ion doni, ne ricevi. Amo estas la arto ion produkti per la kapabloj de la alia. Por tio oni bezonas de la alia respekton kaj simpation. Tion oni povas ĉiam akiri. La troa deziro esti amata malmulte rilatas kun vera amo. Memamo ĉiam havas ion memmortigan."
8,"Helpu al Vikipedio plilon

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model_dir, max_len=512)

In [ ]:
# ds_enc = ds.map(lambda x: tokenizer(x["text"], truncation=True))

In [ ]:
# ds_enc.save_to_disk("data/oscar.eo.ds")

In [ ]:
ds_enc = DatasetDict.load_from_disk("data/oscar.eo.ds")
ds_enc

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'text'],
        num_rows: 974616
    })
})

In [ ]:
tokenizer.decode(ds_enc["train"][0]["input_ids"])

'<s>Ĉu... preĝi | mediti | ricevi instigojn || kanti | muziki || informiĝi | legi | studi || prepari Diservon</s>'

## Baseline model

In [ ]:
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
def baseline_init():
    return RobertaForMaskedLM(config=config)

In [ ]:
sample_ds = ds_enc["train"].train_test_split(train_size=512, test_size=128, seed=42)
bs = 8
logging_steps = sample_ds["train"].num_rows // bs // 4

training_args = TrainingArguments(
    output_dir="models/esperberto",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=bs,
    prediction_loss_only=True,
    evaluation_strategy="steps",
    disable_tqdm=False,
    logging_steps=logging_steps
)

trainer = Trainer(
    model_init=baseline_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=sample_ds["train"],
    eval_dataset=sample_ds["test"]
)

trainer.train();

Loading cached split indices for dataset at data/oscar.eo.ds/train/cache-8b96f05ff2fc9096.arrow and data/oscar.eo.ds/train/cache-e43a2b26405c9c65.arrow


Step,Training Loss,Validation Loss
16,10.374800,9.909284
32,9.829200,9.645468
48,9.572800,9.329382
64,9.437400,9.316128


## Custom model

Goal: implement RoBERTa LM from scratch :) Remove as much boilerplate as possible while maintaining compatibility with the trainer.

In [ ]:
from transformers.models.roberta.modeling_roberta import (
    RobertaPreTrainedModel, RobertaLMHead, RobertaLayer)
from transformers.modeling_outputs import BaseModelOutput

In [ ]:
def create_position_ids_from_input_ids(input_ids, padding_idx, past_key_values_length=0):
    """
    Replace non-padding symbols with their position numbers. Position numbers begin at padding_idx+1. Padding symbols
    are ignored. This is modified from fairseq's `utils.make_positions`.

    Args:
        x: torch.Tensor x:

    Returns: torch.Tensor
    """
    # The series of casts and type-conversions here are carefully balanced to both work with ONNX export and XLA.
    mask = input_ids.ne(padding_idx).int()
    incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask
    return incremental_indices.long() + padding_idx

In [ ]:
class MyRobertaEmbeddings(nn.Module):
    """
    Same as BertEmbeddings with a tiny tweak for positional embeddings indexing.
    
    LT: For some reason, removing the token_type_embeddings produces small numerical differences in the losses - safe to remove?
    """

    # Copied from transformers.models.bert.modeling_bert.BertEmbeddings.__init__
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")

        # End copy
        self.padding_idx = config.pad_token_id
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size, padding_idx=self.padding_idx
        )

    def forward(
        self, input_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0
    ):

        # Create the position ids from the input token ids. Any padded tokens remain padded.
        position_ids = create_position_ids_from_input_ids(
            input_ids, self.padding_idx, past_key_values_length
        ).to(input_ids.device)

        input_shape = input_ids.size()


        token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)


        inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = inputs_embeds + token_type_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
class MyRobertaEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([RobertaLayer(config) for _ in range(config.num_hidden_layers)])

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=False,
        output_hidden_states=False,
    ):
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if getattr(self.config, "gradient_checkpointing", False) and self.training:

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )

In [ ]:
class MyRobertaModel(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.embeddings = MyRobertaEmbeddings(config)
        self.encoder = MyRobertaEncoder(config)
        self.init_weights()
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
    ):
        input_shape = input_ids.size()
        batch_size, seq_length = input_shape

        device = input_ids.device 

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask = self.get_extended_attention_mask(attention_mask, input_shape, device)


        embedding_output = self.embeddings(
            input_ids=input_ids,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
        )
        sequence_output = encoder_outputs.last_hidden_state
        

        return BaseModelOutput(
            last_hidden_state=sequence_output,
        )

In [ ]:
class MyRobertaForMaskedLM(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.roberta = MyRobertaModel(config)
        self.lm_head = RobertaLMHead(config)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
    ):

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
        )
        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        loss_fct = CrossEntropyLoss()
        masked_lm_loss = loss_fct(
            prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

        return MaskedLMOutput(
            loss=masked_lm_loss,
            logits=prediction_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
def model_init():
    return MyRobertaForMaskedLM(config=config)

In [ ]:
# reference values

# Step	Training Loss	Validation Loss
# 16	10.272600	9.798569
# 32	9.720300	9.541902
# 48	9.513300	9.259138
# 64	9.403300	9.244128

In [ ]:
def run_trainer():
    trainer = Trainer(
    model_init=model_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=sample_ds["train"],
    eval_dataset=sample_ds["test"]
    )
    
    trainer.train()
    
run_trainer()

Step,Training Loss,Validation Loss
16,10.272600,9.798569
32,9.720300,9.541902
48,9.513300,9.259138
64,9.403300,9.244128
